In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
from scipy.signal import find_peaks
from scipy.optimize import curve_fit
from scipy.ndimage import gaussian_filter1d

sys.path.insert(0, "../")

import warnings
warnings.filterwarnings('ignore')

from michael import janet

2022-10-10 13:01:03.351058: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Hi there. Welcome to the good place.


In [2]:
uni = pd.read_csv("/Users/oliver hall/Research/unicorn/data/unicorn.csv", index_col=0)

In [3]:
index = 1

In [4]:
j = janet.boot(uni, index=index, use_prior=False)
j.prepare_data()

Already have data downloaded for Gaia ID 2174289161681454720.
If you want to check for new data, run `janet.update()`.
Removing 186 bad data points (out of 1121) using the TESS provided QUALITY array
Summing over 2 x 2 pixel lightcurves. Weighting=median
Removing 4 bad data points (out of 1130) using the TESS provided QUALITY array
Summing over 2 x 2 pixel lightcurves. Weighting=median


Running pixels in aperture: 100%|████████████| 300/300 [00:01<00:00, 294.57it/s]


In [5]:
j.get_rotation(period_range=(1, 10))

### Running Simple Astropy Lomb-Scargle on Sector 16-17 on star 2174289161681454720 ###
### Completed Simple Astropy Lomb-Scargle for Sector 16-17 on star 2174289161681454720 ###
### Running Wavelet Estimation for Sector 16-17 on star 2174289161681454720 ###


100%|█████████████████████████████████████████| 152/152 [00:02<00:00, 63.51it/s]


### Completed Wavelet Estimation for Sector 16-17 on star 2174289161681454720 ###
### Running Composite ACF estimation for Sector 16-17 on star 2174289161681454720 ###
### Completed Composite ACF estimation for Sector 16-17 on star 2174289161681454720 ###
### Running ACF Estimation on star 2174289161681454720 ###
### Completed ACF Estimation for Sector 16-17 on star 2174289161681454720 ###


In [ ]:
j.validate_rotation()
j.view()

*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2D array with a single row if you intend to specify the same RGB or RGBA value for all points.


Plot lightcurves

In [ ]:
ax = j.void['clc_16-17'].plot(label='eleanor')

pls = ['c','raw','pca','corn','r', 'cpm']
for pl in pls:
    j.void[f'{pl}lc_16-17'].plot(ax=ax, label=pl)
ax.legend(fontsize=10)

In [ ]:
ax = j.void['clc_16-17'].to_periodogram(maximum_period=27).plot(label='eleanor', view='period')

pls = ['c','raw','pca','corn','r', 'cpm']
for pl in pls:
    j.void[f'{pl}lc_16-17'].to_periodogram(maximum_period=27).plot(ax=ax, label=pl,view='period', scale='log')
    ax.axvline(j.void[f'{pl}lc_16-17'].to_periodogram(maximum_period=27).period_at_max_power.value)
    print(j.void[f'{pl}lc_16-17'].to_periodogram(maximum_period=27).period_at_max_power)
ax.legend(fontsize=10)